In [ ]:
import time
import csv
import nltk
import re
import requests
import pandas as pd
import numpy as np
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

In [3]:
# only once run for download WordNet or update
import nltk
# nltk.download('wordnet', download_dir='./')
nltk.data.path.append('../')
from nltk.corpus import wordnet as wn

In [4]:
# inherit features from Gold Standard dataset
GS_all_agreed = pd.read_csv('../sampled_count/GS_All_Agreed.csv', index_col=0)
features_target = ['Synsets','domain_x',
                   'nrdirhypers_x',
                   'nrhypos_x',
                   'nrpartrels_normalised_x',
                   'depthfromtopsynset_normalised_x',
                   'glosslength_normalised_x',
                   'minwordlength_x',
                   'nroflemmas_x',
                   'polyscore_max_x',
                   'vote_x']
GS_adopt = GS_all_agreed[features_target]

GS_adopt

,Synsets,domain_x,nrdirhypers_x,nrhypos_x,nrpartrels_normalised_x,depthfromtopsynset_normalised_x,glosslength_normalised_x,minwordlength_x,nroflemmas_x,polyscore_max_x,vote_x
0,Synset('adjustable_wrench.n.01'),tool,1,7,0.0,1.012903,0.563173,17,2,1,nb
1,Synset('allen_wrench.n.01'),tool,1,0,0.0,1.012903,0.391092,12,1,1,nb
2,Synset('alligator_wrench.n.01'),tool,1,0,0.0,1.012903,1.517437,16,1,1,nb
3,Synset('awl.n.01'),tool,1,2,15.7,0.911613,0.985552,3,1,1,b
4,Synset('backsaw.n.01'),tool,1,0,0.0,1.114194,1.110701,7,2,1,nb
...,...,...,...,...,...,...,...,...,...,...,...
834,Synset('ballet_skirt.n.01'),garm,1,0,0.0,0.947552,0.578283,4,2,2,nb
835,Synset('mess_jacket.n.01'),garm,1,0,0.0,1.158120,1.652238,11,3,1,nb
836,Synset('long_johns.n.01'),garm,1,0,0.0,1.052836,0.479149,10,1,1,nb
837,Synset('undies.n.01'),garm,1,0,0.0,1.158120,0.280880,6,1,1,nb


In [5]:
# function to get the frequency of a synset in google ngrams
corpora = dict(eng_2019=26, eng_us_2012=17, eng_us_2009=5, eng_gb_2012=18, eng_gb_2009=6,
               chi_sim_2012=23, chi_sim_2009=11, eng_2012=15, eng_2009=0,
               eng_fiction_2012=16, eng_fiction_2009=4, eng_1m_2009=1,
               fre_2012=19, fre_2009=7, ger_2012=20, ger_2009=8, heb_2012=24,
               heb_2009=9, spa_2012=21, spa_2009=10, rus_2012=25, rus_2009=12,
               ita_2012=22)
def ngram_mean_by_year_sleep(synset, corpus, startYear, endYear, smoothing, caseInsensitive):
    global counter
    m = 0
    synset = wn.synset(synset[8:-2])
    for i in synset.lemmas():
        query = i.name()
        query = query.replace('_',' ') # recovery phrase from _
        params = dict(content=query, year_start=startYear, year_end=endYear,
                      corpus=corpora[corpus], smoothing=smoothing,
                      case_insensitive=caseInsensitive)
        if params['case_insensitive'] is False:
            params.pop('case_insensitive')
        if '?' in params['content']:
            params['content'] = params['content'].replace('?', '*')
        if '@' in params['content']:
            params['content'] = params['content'].replace('@', '=>')
        ## set a counter
        counter += 1
        if counter == 73:
            time.sleep(580)
            counter = 0
        req = requests.get('https://books.google.com/ngrams/graph', params=params)
        res = re.findall('ngrams.data = (.*?);\\n', req.text)
        if res:
            data = {qry['ngram']: qry['timeseries'] for qry in literal_eval(res[0])}
            df = pd.DataFrame(data)
            df.insert(0, 'year', list(range(startYear, endYear + 1)))
        else:
            df = pd.DataFrame()
        if df.shape[1] > 1: # if the query exists in google ngrams.
            ngramsum = df.iloc[:,1].sum(axis = 0, skipna = True) # sum of percentage of the lemmas each year
            # time.sleep(10) # to avoid defense from the google ngram server
            m += ngramsum
        else:
            m += 0 # irrelevant, obviously.
    return m/(endYear-startYear+1)

In [16]:
counter = 0
ngram_mean_by_year_sleep("Synset('awl.n.01')", 'eng_2019', 2009, 2019, 0, True)

2.8056081676068473e-07

In [7]:
counter = 0

GS_adopt['ngram_2009_2019'] = GS_adopt['Synsets'].apply(lambda syn: ngram_mean_by_year_sleep(syn, 'eng_2019', 2009, 2019, 0, True))

GS_adopt.to_csv('./features_google_ngram.csv', index=False)

KeyboardInterrupt: 

In [6]:
def ngram_mean_by_year(synset, corpus, startYear, endYear, smoothing, caseInsensitive):
    m = 0
    synset = wn.synset(synset[8:-2])
    for i in synset.lemmas():
        query = i.name()
        query = query.replace('_', ' ')  # recovery phrase from _
        params = dict(content=query, year_start=startYear, year_end=endYear,
                      corpus=corpora[corpus], smoothing=smoothing,
                      case_insensitive=caseInsensitive)
        if params['case_insensitive'] is False:
            params.pop('case_insensitive')
        if '?' in params['content']:
            params['content'] = params['content'].replace('?', '*')
        if '@' in params['content']:
            params['content'] = params['content'].replace('@', '=>')
        req = requests.get('https://books.google.com/ngrams/graph', params=params)
        res = re.findall('ngrams.data = (.*?);\\n', req.text)
        if res:
            data = {qry['ngram']: qry['timeseries'] for qry in literal_eval(res[0])}
            df = pd.DataFrame(data)
            df.insert(0, 'year', list(range(startYear, endYear + 1)))
        else:
            df = pd.DataFrame()
        if df.shape[1] > 1:  # if the query exists in google ngrams.
            ngramsum = df.iloc[:, 1].sum(axis=0, skipna=True)  # sum of percentage of the lemmas each year
            time.sleep(10)  # to avoid defense from the google ngram server
            m += ngramsum
        else:
            m += 0  # irrelevant, obviously.
    return m / (endYear - startYear + 1)


In [ ]:
# check time
# original
original_start = time.time()
GS_adopt['ngram_2009_2019_original_sleep'] = GS_adopt['Synsets'].apply(lambda syn: ngram_mean_by_year(syn, 'eng_2019', 2009, 2019, 0, True))
original_stop = time.time()
original_period = original_stop - original_start

string = 'original sleep time is ' + str(original_period) + '\n'
with open('time_checker.txt', 'a+') as f:
    f.write(string)

time.sleep(580)
update_start = time.time()
counter = 0
GS_adopt['ngram_2009_2019_updata_sleep'] = GS_adopt['Synsets'].apply(lambda syn: ngram_mean_by_year_sleep(syn, 'eng_2019', 2009, 2019, 0, True))
update_stop = time.time()
update_period = update_stop - update_start
string = 'update sleep time is ' + str(update_period) + '\n'
with open('time_checker.txt', 'a+') as f:
    f.write(string)

GS_adopt.to_csv('./features_google_ngram.csv', index=False)

In [9]:
new_GS_adppt = pd.read_csv('./features_google_ngram.csv', index_col=0)

new_GS_adppt

,domain_x,nrdirhypers_x,nrhypos_x,nrpartrels_normalised_x,depthfromtopsynset_normalised_x,glosslength_normalised_x,minwordlength_x,nroflemmas_x,polyscore_max_x,vote_x,ngram_2009_2019_original_sleep,ngram_2009_2019_updata_sleep
Synsets,,,,,,,,,,,,
Synset('adjustable_wrench.n.01'),tool,1,7,0.0,1.012903,0.563173,17,2,1,nb,2.460615e-08,2.460615e-08
Synset('allen_wrench.n.01'),tool,1,0,0.0,1.012903,0.391092,12,1,1,nb,1.601034e-08,1.601034e-08
Synset('alligator_wrench.n.01'),tool,1,0,0.0,1.012903,1.517437,16,1,1,nb,1.236022e-10,1.236022e-10
Synset('awl.n.01'),tool,1,2,15.7,0.911613,0.985552,3,1,1,b,2.805608e-07,2.805608e-07
Synset('backsaw.n.01'),tool,1,0,0.0,1.114194,1.110701,7,2,1,nb,1.135172e-08,1.135172e-08
...,...,...,...,...,...,...,...,...,...,...,...,...
Synset('ballet_skirt.n.01'),garm,1,0,0.0,0.947552,0.578283,4,2,2,nb,5.498723e-07,5.498723e-07
Synset('mess_jacket.n.01'),garm,1,0,0.0,1.158120,1.652238,11,3,1,nb,1.454007e-08,1.454007e-08
Synset('long_johns.n.01'),garm,1,0,0.0,1.052836,0.479149,10,1,1,nb,8.935307e-08,8.935307e-08
